In [10]:
from keras.callbacks import TensorBoard

In [2]:
import pandas as pd
import numpy as np
import scipy
import sys
import imp

In [3]:
from process_data import load_data,get_rank_of_size
from data_util import unpack_news_data,data_generator
import models
import callbacks
imp.reload(models)
imp.reload(callbacks)

<module 'callbacks' from '/home/beginner/code/stock/callbacks.py'>

In [4]:
rank = get_rank_of_size()
rank.remove('XOM')
rank.remove('GOOG')

In [5]:
data = load_data(rank[:20],encoder_kind='Bert')

data of FB is loaded
data of AAPL is loaded
data of GOOGL is loaded
data of AMZN is loaded
data of BAC is loaded
data of JPM is loaded
data of MSFT is loaded
data of C is loaded
data of WFC is loaded
data of TSLA is loaded
data of GE is loaded
data of TWTR is loaded
data of BABA is loaded
data of INTC is loaded
data of WMT is loaded
data of IBM is loaded
data of CVX is loaded
data of VZ is loaded
data of T is loaded
data of DIS is loaded


In [ ]:
print(rank)

In [ ]:
# sparse matrix
data[rank[0]]['train'][0][0][0].toarray().shape

# TEXTUAL MODEL

In [57]:
model = models.build_textual_model()

In [58]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7, 50, 768)        0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 50, 768)        0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 7, 768)            7700      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 7, 100)            76900     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 7, 100)            45300     
_________________________________________________________________
attention_layer_2 (Attention (None, 100)               1020      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
__________

In [ ]:
b_size = 64
x_train = np.concatenate([data[key]['train'][0] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
x_val = unpack_news_data(np.concatenate([data[key]['val'][0] for key in data],axis=0))
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='model_log/textual/')]
steps = (len(x_train)+b_size-1)//b_size

In [ ]:
model.fit_generator(generator=data_generator(b_size,(x_train,y_train),unpack_news_data),
                    samples_per_epoch=steps,
                    validation_data=(x_val,y_val),
                    callbacks = cbs,
                    epochs=20,
                    verbose=1,
                   )

# NUMERICAL MODEL

In [ ]:
nmodel = models.build_numerical_model()

In [ ]:
print(nmodel.summary())

In [ ]:
b_size = 64
x_train = np.concatenate([data[key]['train'][1] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
x_val = np.concatenate([data[key]['val'][1] for key in data],axis=0)
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='model_log/numerical/')]
steps = (len(x_train)+b_size-1)//b_size

In [ ]:
nmodel.fit_generator(generator=data_generator(b_size,(x_train,y_train)),
                    samples_per_epoch=steps,
                    validation_data=(x_val,y_val),
                    callbacks = cbs,
                    epochs = 20,
                    verbose=1,
                   )

# HYBRID MODEL

In [6]:
hmodel = models.build_hybrid_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
b_size = 64
x1_train = np.concatenate([data[key]['train'][0] for key in data],axis=0)
x2_train = np.concatenate([data[key]['train'][1] for key in data],axis=0)
x1_val = unpack_news_data(np.concatenate([data[key]['val'][0] for key in data],axis=0))
x2_val = np.concatenate([data[key]['val'][1] for key in data],axis=0)
y_val = np.concatenate([data[key]['val'][2] for key in data],axis=0)
y_train = np.concatenate([data[key]['train'][2] for key in data],axis=0)
cbs = [TensorBoard(log_dir='num_log/hybrid/')]
steps = (len(x1_train)+b_size-1)//b_size

In [9]:
hmodel.fit_generator(generator=data_generator(b_size,(x1_train,x2_train,y_train),unpack_news_data),
                    samples_per_epoch=steps,
                    validation_data=([x1_val,x2_val],y_val),
                    callbacks = cbs,
                    epochs=10,
                    verbose=1,
                   )

Instructions for updating:
Use tf.cast instead.


/home/beginner/anaconda3/envs/deeplearning/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., validation_data=([array([[..., callbacks=[<keras.ca..., epochs=10, verbose=1, steps_per_epoch=356)`
  


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
356/356 [==============================] - 79s 222ms/step - loss: 0.6900 - acc: 0.5398 - val_loss: 0.7173 - val_acc: 0.3750
Epoch 2/10
356/356 [==============================] - 73s 205ms/step - loss: 0.6802 - acc: 0.5642 - val_loss: 0.6958 - val_acc: 0.5375
Epoch 3/10
356/356 [==============================] - 73s 206ms/step - loss: 0.6577 - acc: 0.6084 - val_loss: 0.8389 - val_acc: 0.4223
Epoch 4/10
356/356 [==============================] - 74s 207ms/step - loss: 0.6203 - acc: 0.6526 - val_loss: 0.7298 - val_acc: 0.5554
Epoch 5/10
356/356 [==============================] - 73s 206ms/step - loss: 0.5598 - acc: 0.7068 - val_loss: 0.7979 - val_acc: 0.5429
Epoch 6/10
356/356 [==============================] - 73s 205ms/step - loss: 0.4908 - acc: 0.7583 - val_loss: 1.1770 - val_acc: 0.4366
Epoch 7/10
356/356 [==============================] - 74s 208ms/step - loss: 0.4427 - acc: 0.7873 - val_loss: 1.

# Random Guess

In [ ]:
gmodel = models.build_guess_model()

In [ ]:
x_train = np.concatenate([np.random.randint(2, size=(data[key]['val'][2].shape[0],1)) for key in data])
y_train = np.concatenate([data[key]['val'][2] for key in data])

In [ ]:
gmodel.fit(x=x_train,y=y_train,verbose=1,epochs=30,batch_size=32)

In [ ]:
result_dict = {}
for code in rank[:20]:
    result = hmodel.evaluate(x=[unpack_news_data(data[code]['test'][0]),data[code]['test'][1]],
                             y=data[code]['test'][2],batch_size=32)
    result_dict[code] = result[1]
    if 'avg' not in result_dict:
        result_dict['avg'] = 0
    result_dict['avg'] += result[1]
result_dict['avg'] /= 20
print(result_dict)

In [59]:
from keras import backend as K
K.clear_session()